Откроем файл для чтения данных.

In [1]:
import pandas as pd
data = pd.read_csv('2.csv', delimiter = '\t')
data.head(5)

,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0


Посмотрим информацию о считанных данных.

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   login   250000 non-null  object
 1   uid     250000 non-null  int64 
 2   docid   250000 non-null  int64 
 3   jud     250000 non-null  int64 
 4   cjud    250000 non-null  int64 
 5   errors  250000 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 11.4+ MB


Удалим дублирующиеся записи, если они есть.

In [17]:
data.drop_duplicates()
data.shape

(250000, 6)

Проверим наличие NaN значений в таблице.

In [18]:
data.isnull().values.any()

False

Покажем количество ошибочных оценок.

In [34]:
sum_error = data['jud'].compare(data['cjud'])
sum_error.shape

(39678, 2)

Поскольку оценки бинарные, то для определения ошибки асессора воспользуемся следующим методом:
1. Вычтем из значения столбцов jud и cjud. Если асессор выполнил задание правильно,
    то ответ будет 0, если нет, то 1 или -1.
2. Заменим -1 на 1.

In [35]:
errors = data.jud - data.cjud
data['errors'] = errors
data['errors'].replace({-1: 1}, inplace=True)
data

,login,uid,docid,jud,cjud,errors
0,assessor158,158,0,0,0,0
1,assessor238,238,0,0,0,0
2,assessor488,488,0,0,0,0
3,assessor136,136,0,0,0,0
4,assessor300,300,0,0,0,0
...,...,...,...,...,...,...
249995,assessor208,208,49999,1,1,0
249996,assessor139,139,49999,1,1,0
249997,assessor333,333,49999,1,1,0
249998,assessor160,160,49999,1,1,0


Для аггрегации данных напишем функцию, которая будет осуществлять подсчёт ошибок и количество обработанных документов каждым пользователем,
а также процент ошибок.

In [36]:
def summary(x):
    result = {
        'errors_sum': x['errors'].sum(),
        'doc_count': x['docid'].count(),
        'percent_errors': x['errors'].sum() / x['docid'].count() * 100,
    }
    return pd.Series(result).round(0)

Сгруппируем и отсортируем полученные данные по убыванию. 

In [37]:
data.groupby(['uid']).apply(summary).sort_values(by=['percent_errors'], ascending=False)

,errors_sum,doc_count,percent_errors
uid,,,
56,236.0,411.0,57.0
3,230.0,426.0,54.0
234,51.0,99.0,52.0
390,214.0,412.0,52.0
118,205.0,391.0,52.0
...,...,...,...
528,32.0,395.0,8.0
450,32.0,399.0,8.0
63,32.0,409.0,8.0


Расчёт процента ошибок явлется очень субъективной метрикой. Для более качественной оценки, лучше использовать такие показатели, как Accuracy, Sensitivity, Precision